In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Beta

# Teaching a quadruped to walk

Time to try out the learning algorithms that you just implemented on a more difficult problem. The WalkerEnv implements a quadruped robot kind-of thing, see for yourself. The goal is to move in the $x$ direction as fast and as far as possible.

Your goal is to implement a class `WalkerPolicy` with function `determine_actions()` just like the StochasticPolicy we used earlier to control the pendulum. Below is a template of this class, but feel free to alter it however you want. The only important thing is the `determine_actions()` function!

After you implement it, copy `WalkerPolicy` into a separate file `WalkerPolicy.py` that you will upload to BRUTE together with the (optional) learned weights in a zip file. How the policy is implemented is up to you! You are constrained to only the libraries we used so far though, such as torch, numpy etc..

You will get some free points just for uploading a working policy (irrelevant of the performance). Further 2 points will be awarded for successfully traversing a small distance in the x direction.


# Hints

There is no single easy way of doing this, but here are some suggestions on what you could try to improve your policy:

1. This problem is much more difficult, than balancing a pendulum. It is a good idea to use a bit larger network than for the pendulum policy.

2. You can also try to use a different optimizer, such as Adam and play with the hyperparameters.

3. Using a neural network to compute the normal distribution scale $\sigma$ can lead to too much randomness in the actions (i.e. exploration). You can use a fixed $\sigma$ instead, or replace it with a learnable `torch.Parameter` initialized to some small constant. Make sure, you run it through an exponential, or softplus function to ensure $\sigma$ is positive.

4. The exploration can also be reduced by penalizing the variance of the action distribution in an additional loss term.

5. If you see some undesirable behaviour, you can tweak the reward function to penalize it. Even though the $x$ distance is all we care about, adding extra terms to the reward can help guide the learning process (This is known as reward shaping). Simply define a reward function mapping the state $s_{t+1}$ and action $a_t$ to a scalar reward $r_t$ and put it in the config dictionary under the key `'reward_fcn'`. See the `WalkerEnv` class for the implementation of the default reward.

6. Using the normal distribution on a bounded action space can lead to certain problems caused by action clipping. This can be mitigated by using a different distribution, such as the Beta distribution. See the `torch.distributions.beta` module for more information. (Note that Beta distribution is defined on the interval [0,1] and works better with parameters $\alpha,\beta \geq 1$.)


In [2]:
# If you cannot run with the visualization, you can set this to False
VISUALIZE = True
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### import self-made modules

In [3]:
from environment.WalkerEnv import WalkerEnv
from WalkerPolicy import WalkerPolicy
import solution

### Define reward function

In [4]:
# DETECT FLIPPING
def rotate_vector_by_quaternion(v, q):
    """
    Rotate the 3D vector v by the quaternion q (w, x, y, z).
    Returns the rotated vector as a numpy array of shape (3,).
    
    The quaternion must be normalized. If not, we do it anyway inside.
    """
    # unpack
    w, x, y, z = q
    # normalize q just in case
    norm_q = np.sqrt(w*w + x*x + y*y + z*z)
    w, x, y, z = w/norm_q, x/norm_q, y/norm_q, z/norm_q

    # v as quaternion: (0, v.x, v.y, v.z)
    vx, vy, vz = v

    # quaternion product q*v
    #  result = q*v = ( w*0 - x*vx - y*vy - z*vz,
    #                   w*vx + x*0 + y*vz - z*vy,
    #                   w*vy - x*vz + y*0 + z*vx,
    #                   w*vz + x*vy - y*vx + z*0 )
    #  but let's do it in a simpler function:
    #  p = (0, vx, vy, vz)
    #  q * p = (w*0 - x*vx - y*vy - z*vz,
    #           w*vx + x*0 + y*vz - z*vy,
    #           w*vy - x*vz + y*0 + z*vx,
    #           w*vz + x*vy - y*vx + z*0)

    # for clarity:
    qw = w
    qx = x
    qy = y
    qz = z

    # first multiply q * p
    tw = - (qx*vx + qy*vy + qz*vz)
    tx =   qw*vx + qy*vz - qz*vy
    ty =   qw*vy - qx*vz + qz*vx
    tz =   qw*vz + qx*vy - qy*vx

    # then multiply (q*p)*q^* => (tw, tx, ty, tz) * (qw, -qx, -qy, -qz)
    #  result is also a quaternion
    # w' = tw*qw - tx*qx - ty*qy - tz*qz
    rw = tw*qw - tx*qx - ty*qy - tz*qz
    rx = tw*(-qx) + tx*qw + ty*(-qz) - tz*(-qy)
    ry = tw*(-qy) - tx*(-qz) + ty*qw + tz*(-qx)
    rz = tw*(-qz) + tx*(-qy) - ty*(-qx) + tz*qw

    # The rotated vector is the (rx, ry, rz) part
    return np.array([rx, ry, rz], dtype=np.float32)



DISTANCE_MULTIPLIER = 0
VELOCITY_MULTIPLIER = 0
ACTION_PENALTY_MULTIPLIER = 0
STABILITY_PENALTY_MULTIPLIER = 1

def walker_reward(state, action):
    pos = state[:15]  # first 15 elements of state vector are generalized coordinates [xyz, quat, joint_angles]
    vel = state[15:]  # last 14 elements of state vector are generalized velocities [xyz_vel, omega, joint_velocities]
    x_velocity = vel[0]  # this is the x axis velocity
    x_distance = pos[0]  # this is the x axis position
    stability_penalty = np.sum(np.abs(vel[1:3]))  # Penalize y and z velocities
    action_penalty = np.sum(np.square(action))  # Penalize large actions
    return (x_distance * DISTANCE_MULTIPLIER + 
            x_velocity * VELOCITY_MULTIPLIER - 
            STABILITY_PENALTY_MULTIPLIER * stability_penalty - 
            ACTION_PENALTY_MULTIPLIER * action_penalty)



# ------------------------------------------------------------------------------------------------
# partial reward shaping for the Walker environment
# ------------------------------------------------------------------------------------------------
def sample_trajectories(env, policy, T, device='cpu'):
    obs_list = env.reset()[0]  
    N = len(obs_list)
    state_dim = len(obs_list[0])
    action_dim = 8

    flip_penalty = 0.0

    obs = torch.tensor(np.stack(obs_list), dtype=torch.float32, device=device)
    states  = torch.zeros((T+1, N, state_dim), device=device)
    actions = torch.zeros((T,   N, action_dim), device=device)
    rewards = torch.zeros((T,   N),            device=device)

    states[0] = obs

    local_up = np.array([0.0, 0.0, 1.0], dtype=np.float32)

    for t in range(T):
        with torch.no_grad():
            a, _, _, _ = policy(states[t])

        actions[t] = a
        a_np = a.cpu().numpy().reshape(-1)

        next_obs_list, env_reward_list = env.vector_step(a_np)
        next_obs = torch.tensor(np.stack(next_obs_list), dtype=torch.float32, device=device)

        custom_rewards = []
        for i in range(N):
            # ------------------------- PENALISE FLIPPING -------------------------
            # parse the quaternion:
            q_wxyz = next_obs_list[i][3:7]  # [w, x, y, z]
            # rotate local_up:
            up_world = rotate_vector_by_quaternion(local_up, q_wxyz)
            # if up_world.z < 0 => flipped
            if up_world[2] < -0.9:
                flip_penalty = -5.0
                #print(f"Robot {i} flipped at time {t}") # debug if this code works
            # -------------------------REWARD ------------------------
            x_pos = next_obs_list[i][0]
            position_reward = 1.0 * x_pos
            x_vel = next_obs_list[i][15]
            forward_reward = 10.0 * x_vel
            # ---------------------- PENALISE GOING BACKWARDS MORE --------------------
            if x_vel < 0.0:
                forward_reward *= 1.5
            # ---------------------- PENALISE BEING STILL --------------------
            if np.abs(x_vel) < 0.05:
                forward_reward *= 0

            new_r = forward_reward + position_reward + 0.1*env_reward_list[i] if flip_penalty == 0.0 else flip_penalty + position_reward
            custom_rewards.append(new_r)
            flip_penalty = 0.0

        r = torch.tensor(custom_rewards, dtype=torch.float32, device=device)
        states[t+1] = next_obs
        rewards[t]  = r

    return states, actions, rewards



## Train loop
### helper functions

In [5]:
def compute_gae_no_done(rewards, values, gamma=0.99, lam=0.95):
    """
    rewards: [T, N]
    values:  [T+1, N]
    Returns:
      advantages:   [T, N]
      value_target: [T, N] = advantages + values[:T]
    """
    T, N = rewards.shape
    advantages = torch.zeros((T, N), dtype=torch.float32, device=rewards.device)
    last_gae = torch.zeros((N,), dtype=torch.float32, device=rewards.device)

    for t in reversed(range(T)):
        delta = rewards[t] + gamma*values[t+1] - values[t]
        last_gae = delta + gamma*lam*last_gae
        advantages[t] = last_gae

    value_target = advantages + values[:-1]  # shape [T, N]
    return advantages, value_target

def ppo_loss(p_ratios, advantages, epsilon=0.2):
    """
    p_ratios:   exp(log_pi(a|s) - log_pi_old(a|s)) [T*N]
    advantages: [T*N]
    """
    clipped = torch.clamp(p_ratios, 1.0 - epsilon, 1.0 + epsilon) * advantages
    return -torch.min(p_ratios * advantages, clipped).mean()

def value_loss(value_preds, value_targets):
    """
    Simple MSE on the value function
    """
    return 0.5 * (value_preds - value_targets).pow(2).mean()


def visualize_policy_progress(
    epoch: int,
    policy,
    device: str = "cpu",
    steps: int = 256,
    vis_env_config: dict = None
):
    """
    Visualizes the policy's behavior by running a short rollout 
    in a single environment with visualization turned on.
    
    Args:
        epoch (int): current training epoch (for logging/title)
        policy: your actor-critic policy (with .determine_actions or forward)
        device (str): "cpu" or "cuda"
        steps (int): how many steps to roll out for visualization
        vis_env_config (dict): environment config dict, e.g. {'N': 1, 'vis': True, ...}
    """

    # 1. Create or reuse an environment specifically for visualization
    #    We'll assume your environment constructor takes a config dict.
    if vis_env_config is None:
        # default minimal config
        vis_env_config = {
            'N': 1,         # single walker
            'vis': True,    # turn on visualization
            'track': 0      # track robot index 0
            # 'reward_fcn': walker_reward, # if needed
        }
    eval_env = WalkerEnv(vis_env_config)

    # 2. Reset environment and prepare for rollout
    obs_list = eval_env.reset()  # returns list of size N=1, each obs is shape (29,)
    obs = torch.tensor(obs_list[0], dtype=torch.float32, device=device).unsqueeze(0)  
    # shape [1, 29]

    # 3. We'll collect x-positions (pos[0]) over time to see how far it travels
    x_positions = []

    # 4. Roll out policy
    for t in range(steps):
        # a) Determine action from the policy
        with torch.no_grad():
            action_tensor = policy.determine_actions(obs)  # shape [1, action_dim]
        action_np = action_tensor.cpu().numpy().reshape(-1)  # shape (action_dim,)

        # b) Step environment
        next_obs_list, reward_list = eval_env.vector_step(action_np)
        # next_obs_list: length=1, each shape(29,)

        # c) Convert next_obs to torch
        next_obs = torch.tensor(next_obs_list[0], dtype=torch.float32, device=device).unsqueeze(0)

        # d) Record x-position for debugging
        x_position = next_obs_list[0][0]  # the x-coord is pos[0]
        x_positions.append(x_position)

        obs = next_obs

    # 5. Close the environment (if it requires manual close)
    eval_env.close()

    # 6. Visualize or print the results
    #    For example, we can plot the x-positions over time
    plt.figure(figsize=(6,4))
    plt.plot(x_positions, label='x position')
    plt.title(f"Policy Visualization at Epoch {epoch}")
    plt.xlabel("Timestep")
    plt.ylabel("X Position")
    plt.legend()
    plt.show()

### Training function

In [6]:
def run_ppo_training_debug(refine=False):
    # ---------------------------------------------------------
    # Hyperparameters
    # ---------------------------------------------------------
    N = 4
    T = 512
    epochs = 500
    gamma = 0.95
    lam   = 0.97
    epsilon = 0.25
    sgd_iters = 5

    actor_lr  = 0.001
    critic_lr = 0.001
    entropy_coef = 0.1
    exploration_penalty_start = 0.0
    exploration_penalty_end   = 0.0
    smoothness_coef = 0.1

    # ---------------------------------------------------------
    # Environment & Policy
    # ---------------------------------------------------------
    config = {
        'N': N,
        'vis': False,
        'track': 0,
        'reward_fcn': walker_reward
    }
    env = WalkerEnv(config)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("[DEBUG] Using device:", device)

    state_dim = 29
    action_dim = 8
    policy = WalkerPolicy(state_dim, action_dim).to(device)

    if refine:
        policy.load_weights()

    # Separate actor & critic params
    actor_params, critic_params = [], []
    for name, param in policy.named_parameters():
        if "actor_network" in name:
            actor_params.append(param)
        else:
            critic_params.append(param)

    actor_optimizer  = optim.Adam(actor_params,  lr=actor_lr)
    critic_optimizer = optim.Adam(critic_params, lr=critic_lr)

    # ---------------------------------------------------------
    # Logging arrays
    # ---------------------------------------------------------
    mean_rewards_list = []
    policy_loss_list  = []
    value_loss_list   = []

    entropies_log      = []
    alpha_min_log      = []
    alpha_max_log      = []
    beta_min_log       = []
    beta_max_log       = []
    actions_min_log    = []
    actions_max_log    = []

    best_reward = 0
    improved = False

    pbar = tqdm(range(epochs), desc="PPO Training with Debug")

    for epoch in pbar:
        # 1) Collect rollouts (using our custom reward logic in sample_trajectories)
        states, actions, rewards = sample_trajectories(env, policy, T, device=device)
        # shapes: 
        #   states:  [T+1, N, state_dim]
        #   actions: [T,   N, action_dim]
        #   rewards: [T,   N]

        # For debugging: let's look at the raw action stats
        all_actions_np = actions.cpu().numpy().reshape(-1, action_dim)
        actions_min_log.append(all_actions_np.min())
        actions_max_log.append(all_actions_np.max())

        with torch.no_grad():
            values_all = policy.value_estimates(states.view(-1, state_dim)).view(T+1, N)
            states_flat  = states[:-1].reshape(T*N, state_dim)
            actions_flat = actions.reshape(T*N, action_dim)
            logp_old = policy.log_prob(actions_flat, states_flat)

        # 3) GAE
        advantages, value_targets = compute_gae_no_done(rewards, values_all, gamma=gamma, lam=lam)
        adv_flat = advantages.view(T*N)
        val_targ_flat = value_targets.view(T*N)

        adv_mean = adv_flat.mean()
        adv_std  = adv_flat.std() + 1e-8
        adv_flat = (adv_flat - adv_mean) / adv_std

        # 4) PPO update
        fraction = epoch / float(epochs)
        exploration_penalty_coef = (exploration_penalty_start 
                                    + fraction*(exploration_penalty_end - exploration_penalty_start))

        for _ in range(sgd_iters):
            # a) Policy update
            actor_optimizer.zero_grad()
            logp = policy.log_prob(actions_flat, states_flat)
            ratio = torch.exp(logp - logp_old)
            L_ppo = ppo_loss(ratio, adv_flat, epsilon=epsilon)

            # distribution stats for debugging
            with torch.no_grad():
                alpha_beta = policy.actor_network(states_flat)
                alpha, beta = torch.chunk(alpha_beta, 2, dim=-1)
                alpha = F.softplus(alpha) + 1.0
                beta  = F.softplus(beta)  + 1.0

                dist = Beta(alpha, beta)
                entropy = dist.entropy().mean()

                entropies_log.append(entropy.item())
                alpha_min_log.append(alpha.min().item())
                alpha_max_log.append(alpha.max().item())
                beta_min_log.append(beta.min().item())
                beta_max_log.append(beta.max().item())

            L_entropy = -entropy_coef * entropy

            # exploration penalty is 0 for now
            variance_each_dim = (alpha * beta) / ((alpha+beta)**2 * (alpha+beta+1.0))
            exploration_var = variance_each_dim.mean()
            L_exploration = exploration_penalty_coef * exploration_var

            # smoothness penalty
            actions_shifted = actions[1:] - actions[:-1]
            action_diff_penalty = (actions_shifted**2).mean()
            L_smoothness = smoothness_coef * action_diff_penalty

            policy_loss = L_ppo + L_exploration + L_smoothness # + L_entropy THIS IS BULLSHIT BECAUSE ENTROPY IS NEGATIVE?????
            policy_loss.backward()
            actor_optimizer.step()

            # b) Critic update
            critic_optimizer.zero_grad()
            new_values_all = policy.value_estimates(states.view(-1, state_dim)).view(T+1, N)
            new_values_t   = new_values_all[:-1].reshape(T*N)
            L_v = value_loss(new_values_t, val_targ_flat)
            L_v.backward()
            critic_optimizer.step()

        # 5) Logging
        mean_reward = rewards.mean().item()
        mean_rewards_list.append(mean_reward)
        policy_loss_list.append(policy_loss.item())
        value_loss_list.append(L_v.item())

        if mean_reward > best_reward:
            best_reward = mean_reward
            policy.save_weights()
            improved = True
            print(f"[DEBUG] New best mean reward: {best_reward:.3f}, model saved.")

        pbar.set_postfix({
            'Epoch': epoch,
            'MeanReward': f"{mean_reward:.3f}",
            'PolicyLoss': f"{policy_loss.item():.4f}",
            'ValueLoss': f"{L_v.item():.4f}",
            'Entropy': f"{entropy.item():.3f}",
            'VarPenalty': f"{exploration_var.item():.3f}",
            'Smoothness': f"{action_diff_penalty.item():.3f}"
        })

        # Optional: visualize every 50 epochs
        if (epoch % 50 == 0 and epoch > 0): #or (improved and epoch > 100) :
            visualize_policy_progress(
                epoch,
                policy,
                device=device,
                steps=T,
                vis_env_config={
                    'N': 1,
                    'vis': True,
                    'track': 0,
                }
            )
            improved = False

    # end training
    env.close()

    # Plot all your logs
    plt.figure(figsize=(15,8))
    plt.subplot(2,3,1)
    plt.plot(mean_rewards_list)
    plt.title("Mean Reward")
    plt.xlabel("Epoch")

    plt.subplot(2,3,2)
    plt.plot(policy_loss_list, color="orange")
    plt.title("Policy Loss")
    plt.xlabel("Epoch")

    plt.subplot(2,3,3)
    plt.plot(value_loss_list, color="green")
    plt.title("Value Loss")
    plt.xlabel("Epoch")

    plt.subplot(2,3,4)
    plt.plot(entropies_log, label="Entropy")
    plt.title("Policy Entropy")
    plt.xlabel("Update step")

    plt.subplot(2,3,5)
    plt.plot(actions_min_log, label="Action min")
    plt.plot(actions_max_log, label="Action max")
    plt.title("Action Range Over Epochs")
    plt.legend()

    plt.subplot(2,3,6)
    plt.plot(alpha_min_log, label="Alpha min", color="red")
    plt.plot(alpha_max_log, label="Alpha max", color="blue")
    plt.title("Alpha Range Over Updates")
    plt.legend()

    plt.tight_layout()
    plt.show()

    return mean_rewards_list, policy_loss_list, value_loss_list


### Train model

In [ ]:
mean_rewards, p_losses, v_losses = run_ppo_training_debug(True)


## Debug how the quadruped moves

In [ ]:
def debug_wave_actions():
    env = WalkerEnv({'N': 1, 'vis': True, 'track': 0})
    obs = env.reset()

    for step in range(500):
        # example: wave each joint from -1 to +1 over time
        wave = np.sin(step * 0.3)  # range ~ [-0.5, 0.5]
        action = np.array([-2, wave, 2, wave, -2.0, 3, 2.0, 3])  # shape (8,)
        obs, rewards = env.vector_step(action)
        local_up = np.array([0.0, 0.0, 1.0], dtype=np.float32)
        q_wxyz = obs[0][3:7]  # [w, x, y, z]
        # rotate local_up:
        up_world = rotate_vector_by_quaternion(local_up, q_wxyz)
            
        # if up_world.z < 0 => flipped
        print (f"step: {step} UW:{up_world[2]}")
        if up_world[2] < -0.9:
            print(f"Robot flipped at time {step}")
        # does the robot's legs wave? do we see any movement?

    env.close()

debug_wave_actions()


## Visualise the trained quadruped

In [ ]:
# This is the configuration for the Walker environment
# N is the number of robots controlled in parallel
# vis is a boolean flag to enable visualization
# !! IMPORTANT track is an integer index to enable camera tracking of a particular robot (indexed by the value of the argument), this is useful when evaluating the performance of the policy after training
# reward_fcn is the reward function that the environment will use to calculate the reward
T = 1000
x = -1000
N = 1

env = WalkerEnv({'N': N, 'vis': VISUALIZE, "track": 0, "reward_fcn": walker_reward})
obs = env.vector_reset()  # shape (4, 29)
policy = WalkerPolicy()
policy.load_weights()

for i in range(T):
    # obs is now shape (4, 29)
    obs_tensor = torch.tensor(obs, dtype=torch.float32)  # shape (N, 29)
    actions_tensor = policy.determine_actions(obs_tensor)  # shape (N, 8)

    # Flatten to (32,) for env.vector_step
    actions_np = actions_tensor.cpu().numpy().reshape(-1)
    obs, reward = env.vector_step(actions_np)  # returns new obs, reward for all N walkers

    # For tracking the maximum x across all walkers:
    # obs is a list of length 4, each a (29,)-shaped array
    for w in range(N):
        x = max(x, obs[w][0])

env.close()
print(f"After {T} steps, the maximum x value reached was {x}")
